In [23]:
import qval as qv
import numpy as np
import random

In [24]:

bits = 4

def quantize_matrix(matrix, n):
    rows, cols = matrix.shape

    qmatrix = [[0 for _ in range(rows)] for _ in range(cols)]

    for i in range(rows):
        for j in range(cols):
            qmatrix[i][j] = qv.quantize(matrix[i][j], n)

    return np.array(qmatrix)

def dequantize_matrix(qmatrix, n):
    rows, cols = qmatrix.shape

    matrix = [[0 for _ in range(rows)] for _ in range(cols)]

    for i in range(rows):
        for j in range(cols):
            matrix[i][j] = qv.dequantize(qmatrix[i][j], n)

    return np.array(matrix)



In [25]:
def qMatMul(m1, m2, n):
    rows = m1.shape[0]
    cols = m2.shape[1]
    muls = m2.shape[0]

    m3 = np.array([[qv.qZERO(n) for _ in range(cols)] for _ in range(rows)], dtype=object)

    for i in range(rows):
        for j in range(cols):
            sum_ij = qv.qZERO(2 * n - 1)
            for k in range(muls):
                prod = qv.qMul(m1[i][k], m2[k][j])
                sum_ij = qv.qAdd(sum_ij, qv.qfit(prod, len(sum_ij)))
                sum_ij = sum_ij[-(2 * n - 1):]
            
            
            m3[i][j] = qv.qfit(sum_ij, n)

    return m3


def MatMul(m1, m2):
    rows = m1.shape[0]
    cols = m2.shape[1]
    muls = m2.shape[0]

    m3 = np.array([[0 for _ in range(cols)] for _ in range(rows)], dtype=object)

    for i in range(rows):
        for j in range(cols):
            sum_ij = 0
            for k in range(muls):
                
                prod = m1[i][k] * m2[k][j]
                sum_ij += prod
            
            m3[i][j] = sum_ij

    return m3

def matrix_mean_error(matrix, qmatrix):
    error_sum = 0
    for i in range(3):
        for j in range(3):
            error_sum += matrix[i][j] - qmatrix[i][j]

    return error_sum / 9

In [32]:
mat1 = np.random.rand(3, 3)
mat2 = np.random.rand(3, 3)

qmat1 = quantize_matrix(mat1, bits)
qmat2 = quantize_matrix(mat2, bits)

qmultmat = qMatMul(qmat1, qmat2, bits)
qmultmat = dequantize_matrix(qmultmat, bits)
print(qmultmat)
multmat = MatMul(mat1, mat2)
print(multmat)
error = matrix_mean_error(multmat, qmultmat)
print(error)

[[0.         0.42857143 0.57142857]
 [0.         0.14285714 0.14285714]
 [0.         0.42857143 0.57142857]]
[[0.22022548141122944 0.6949898520769704 0.6651542032535456]
 [0.10491086164105046 0.21866175948681021 0.2566208308375725]
 [0.18754238139277768 0.6394882922428217 0.5586597383177547]]
0.14005990166069412


In [27]:
multmat = MatMul(mat1, mat2)
multmat

array([[1.0971929201650346, 0.8300955755690755, 0.8578356068287666],
       [1.0486805680999427, 0.7734216607403637, 0.7293805343101658],
       [1.0612144559032246, 0.8435318456286602, 0.9632253186201026]],
      dtype=object)

In [6]:
bits = 8
print(qv.qMax_i(bits))
a = random.uniform(0, 1)
b = random.uniform(0, 1)
print(f"a: {a}, b: {b}, a+b: {a+b}")
qa = qv.quantize(a, bits)
qb = qv.quantize(b, bits)
print(f"qa: {qa}, qb: {qb}, qa+qb: {qv.qMul(qa, qb)}")
print(f"qai: {qv.qToInt(qa)}, qbi: {qv.qToInt(qb)}, (qa+qb)i: {qv.qToInt(qv.qMul(qa, qb))}")
print(f"dqa: {qv.dequantize(qa)}, dqb: {qv.dequantize(qb)}, d(qa+qb): {qv.dequantize(qv.qMul(qa, qb), bits + 2)}")
sum = qv.qAdd(qa, qb)
num = qv.quantize(a + b, bits)
print(qv.qToInt(qv.qMul(qa, qb)))
print(qv.qFromInt(qv.qToInt(qv.qMul(qa, qb)), bits))
print(qv.qAdd(qa, qb))
print(qv.dequantize(qv.qFromInt(qv.qToInt(qv.qMul(qa, qb)), bits), bits))

127
a: 0.3590657458731036, b: 0.2235455496148533, a+b: 0.5826112954879569
qa: 00101110, qb: 00011100, qa+qb: 000010100001000
qai: 46, qbi: 28, (qa+qb)i: 1288
dqa: 0.36220472440944884, dqb: 0.2204724409448819, d(qa+qb): 2.5205479452054793
1288
01111111
001001010
1.0
